In [ ]:
# Setup stuff
!git clone https://github.com/justin73939/ClauseWise.git
%cd /content/ClauseWise
!pip install -r requirements.txt

Cloning into 'ClauseWise'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 50 (delta 11), reused 49 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 1.12 MiB | 12.43 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/ClauseWise


In [ ]:
# Load the model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "ClauseWise/legalbert-clause-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ClauseWise/legalbert-clause-classifier and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import files

# Upload your local CSV (downloaded from Atticus site)
uploaded = files.upload()

Saving master_clauses.csv to master_clauses.csv


In [ ]:
import pandas as pd
import re

# Load the master CSV you uploaded
df = pd.read_csv("data/master_clauses.csv")
print("Shape before flattening:", df.shape)

# Flatten: one clause per row
long_rows_classif = []
long_rows_textgen = []
for _, row in df.iterrows():
    filename = row["Filename"]
    for col in df.columns:
        if col.endswith("-Answer"):  # find every label column
            category = col.replace("-Answer", "").strip()
            text_col = category
            if text_col not in df.columns:
                continue
            text = row[text_col]
            answer = row[col]

            # skip empty entries
            if pd.isna(text) or text in ([], "", None): continue

            # clean stringified lists
            if isinstance(text, str):
                text = text.strip("[]").replace("'", "").strip()
            if isinstance(answer, str):
                answer = answer.strip("[]").replace("'", "").strip()

            # Separate the 8 columns that do not have yes/no answers for the text generator
            if category in ["Document Name", "Parties", "Agreement Date", "Effective Date", "Expiration Date", "Renewal Term", "Notice Period", "Governing Law", "Most Favored Nation"]:
                long_rows_textgen.append({
                  "document_name": filename,
                  "category": category,
                  "text": text,
                  "answer": answer
              })
                continue

            long_rows_classif.append({
                "document_name": filename,
                "category": category,
                "text": text,
                "answer": answer
            })

flat_df_classif = pd.DataFrame(long_rows_classif)
flat_df_textgen = pd.DataFrame(long_rows_textgen)
print("Flattened shape:", flat_df_classif.shape)
print("Flattened shape:", flat_df_textgen.shape)

# Clean up text artifacts
def clean_clause(t):
    if pd.isna(t): return ""
    t = re.sub(r"<omitted>", " ", t)
    t = re.sub(r"\[\*+\]", " ", t)
    t = re.sub(r"_+", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

flat_df_classif["text"] = flat_df_classif["text"].apply(clean_clause)
flat_df_classif["answer"] = flat_df_classif["answer"].apply(clean_clause)

flat_df_textgen["text"] = flat_df_textgen["text"].apply(clean_clause)
flat_df_textgen["answer"] = flat_df_textgen["answer"].apply(clean_clause)

# Drop empties and tiny fragments
flat_df_classif = flat_df_classif.dropna(subset=["text", "category"])
flat_df_classif = flat_df_classif[flat_df_classif["text"].str.len() > 5]
flat_df_classif = flat_df_classif.drop_duplicates(subset=["text", "category"])

flat_df_textgen = flat_df_textgen.dropna(subset=["text", "category"])
flat_df_textgen = flat_df_textgen[flat_df_textgen["text"].str.len() > 5]
flat_df_textgen = flat_df_textgen.drop_duplicates(subset=["text", "category"])

# Save cleaned file
flat_df_classif.to_csv("data/cuad_flattened_classification.csv", index=False)
print("Saved → data/cuad_flattened_classification.csv")
flat_df_textgen.to_csv("data/cuad_flattened_text_generation.csv", index=False)
print("Saved → data/cuad_flattened_text_generation.csv")

#print(flat_df_classif.head(50))
print(flat_df_textgen.head(50))



Shape before flattening: (510, 83)
Flattened shape: (16830, 4)
Flattened shape: (4080, 4)
Saved → data/cuad_flattened_classification.csv
Saved → data/cuad_flattened_text_generation.csv
                                        document_name             category  \
0   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...        Document Name   
1   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...              Parties   
2   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...       Agreement Date   
3   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...       Effective Date   
4   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...      Expiration Date   
5   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...         Renewal Term   
6   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...        Governing Law   
8   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...        Document Name   
9   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...              Parties   
10  EuromediaHoldingsCorp_20070215_

In [ ]:
import pandas as pd

df_class = pd.read_csv("data/cuad_flattened_classification.csv")
print(df_class.shape)
print(df.columns)
print(df.category.value_counts().head())



(6279, 4)
Index(['document_name', 'category', 'text', 'answer'], dtype='object')
category
Parties            501
Agreement Date     455
Governing Law      428
Expiration Date    409
Effective Date     376
Name: count, dtype: int64


In [ ]:
# Push model to huggingface hub
from huggingface_hub import login

# Login (only once per session)
login()

# Push to your namespace
model.push_to_hub("ClauseWise/legalbert-clause-classifier")
tokenizer.push_to_hub("ClauseWise/legalbert-clause-classifier")